# Observation Targets Assessor
This notebook aims to injest a list of source descriptions and observation criteria from the STM effort, applying appropriate metric tools to assess against the critera for a list of mission concept descriptions.

To run this on Google Colab:
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnGBaker/GWI-metrics/blob/main/Notebooks/ObservationTargetsAssessor.ipynb)

### Some basic setup

In [1]:
if 'google.colab' in str(get_ipython()):
    !git clone https://github.com/JohnGBaker/GWI-metrics.git
    src='GWI-metrics/src/'
    datadir='GWI-metrics/Data/'
else:
    src='../src/'
    datadir='../Data/'
!mkdir -p '../plots/'

# Import
import os
from datetime import date
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append(src)
import constants
import metrics
import sources
import concepts
import subsystems
import background
import pandas as pd
from glob import glob
import re
from astropy.coordinates import Angle
from astropy.cosmology import WMAP9 as cosmo
from astropy import units as u

### Define the set of concepts
Here we load in a set of the pre-defined concepts, or you can define your own

In [2]:
missionNames = (
    'LISACBE',
    #'LISASciRDv1',
    'TwinLISA',
    'LISAGrande',
    'LISAU',
    'GoBIGLISA',
    'ALIA',
    'ALIAtwin',
    'GoBIGALIA')
    #'ALIAlowL')
missions=[concepts.menu[mission] for mission in missionNames]

for mission in missions:
    mission=background.add2model(mission)
#model = concepts.LISACBE.copy()
#modelName = model.get('label')
#model = background.add2model(model)     # add galactic background model

### Load the observation target data
Load information about the target sources and the observation quality targets from the STM.
The data are from https://nasa.sharepoint.com/:x:/t/GravitationalWaveImager/EaaeMC7L-2NJpFCQloYbSnoBDZag_cvFWR5_BjRoFAD6Tw?e=uWr4mU

In [3]:
#datadir='../Data/'
files=glob(datadir+'ObservationObjectives*.csv')
files.sort(key=os.path.getmtime)
print('Observation targets files are:',files)
file=files[-1]
print('Observation targets file is:',file)
#Read file
df=pd.read_csv(file,header=1)
print(len(df),'rows read from file.')
if True:
    #Drop empty rows
    for i,row in df.iterrows():
        if row.isnull().all():
            df=df.drop(i)
    #Fill empties in first column with values above
    val=float('nan')
    col='Source class'
    for i,row in df.iterrows():
        if row.isnull()[col]:
            df.loc[i,col]=val
        else:
            val=df.loc[i,col]
#Drop rows without a labeled objective
for i,row in df.iterrows():
    if row.isnull()['provisional POC label']:
        df=df.drop(i)
print(len(df),'rows with a potential observation label.')
#Fill empties with value above
if True:
    for columnName, columnData in df.iteritems():
        val=float('nan')
        nulls=columnData.isnull()
        for i,row in df.iterrows():
            if nulls[i]:
                df.loc[i,columnName]=val
            else:
                val=df.loc[i,columnName]
#derive masses
if True:
    col=2
    df['m1']=float('nan')
    df['m2']=float('nan')
    for i,row in df.iterrows():
        txt=row.iloc[col]
        mtxt=re.search('\s*Msun\s*\+\s*.*Msun',txt)
        #print('::',mtxt)
        mtxt=re.search('(\d+\.*\d*[eE]*[\+-]*\d*)\s*M[Ss]un\s*\+\s*(\d+\.*\d*[eE]*[\+-]*\d*)\s*M[Ss]un',txt)
        #print(':',mtxt)
        m1=None
        if mtxt is not None: 
            try:
                m1=float(mtxt[1])
                #print(m1)
                m2=float(mtxt[2])
                #print(m2)
            except: 
                pass
        if m1 is None:
            df=df.drop(i)
        else:
            df.loc[i,'m1']=m1
            df.loc[i,'m2']=m2
    print(len(df),'rows with masses.')
#derive redshifts/distances
if True:
    col=3
    df['z']=float('nan')
    df['dist']=float('nan')
    for i,row in df.iterrows():
        txt=row.iloc[col]
        ztxt=re.search('z\s*=\s*(\d+\.*\d*[eE]*[\+-]*\d*)',txt)
        z=None
        if ztxt is not None: 
            try:
                z=float(ztxt[1])
                dist=cosmo.luminosity_distance(z).to(u.kpc).value
            except: 
                pass
        if z is None:
            df=df.drop(i)
        else:
            df.loc[i,'z']=z
            df.loc[i,'dist']=dist
    print(len(df),'rows with redshift.')
#derive timecuts
if True:
    col=4
    df['timecut']=float('nan')
    for i,row in df.iterrows():
        txt=row.iloc[col]
        ztxt=re.search('(.*)\s+(before|<)\s+merger',txt)
        z=None
        
        if ztxt is not None:
            try:
                ztxt=ztxt[1]
                if ztxt=='month to week': z=15
                elif ztxt=='year to month': z=180
                elif ztxt=='days': z=3
                elif ztxt=='1 day': z=1
                elif ztxt=='week': z=7
                elif ztxt=='2 years': z=360
            except: 
                pass
        df.loc[i,'timecut']=z
#derive SNRs
if True:
    col=4
    df['SNR']=float('nan')
    for i,row in df.iterrows():
        txt=row.iloc[col]
        ztxt=re.search('SNR\s*=\s*(\d+\.*\d*[eE]*[\+-]*\d*)',txt)
        z=None
        if ztxt is not None: 
            try:
                z=float(ztxt[1])
            except: 
                pass
        df.loc[i,'SNR']=z
#prep CW sources
if True:
    col=0
    df['CW']=False
    df['f0']=float('nan')
    for i,row in df.iterrows():
        txt=row.iloc[col]
        if 'Persistent' in txt: 
            df.loc[i,'CW']=True
            #Also need to f0 (or separation)
            txt=row['freq']
            try:
                ftxt=re.search('(\d+\.*\d*[eE]*[\+-]*\d*)\s+Hz',txt)[1]
                f0=float(ftxt)
                df.loc[i,'f0']=f0
            except:
                print('Failed to get freq from txt "'+txt+'"')
                df=df.drop(i)
#process angular precision
if True:
    col=6
    df['resolution']=float('nan')
    for i,row in df.iterrows():
        txt=row.iloc[col]
        z=None
        #print(txt)
        try:
            #print(Angle(txt))
            z=Angle(txt).arcsecond
        except: 
            pass
        df.loc[i,'resolution']=z
#Drop cases with neither angular precision requirement nor SNR
for i,row in df.iterrows():
    if np.isnan(row['resolution']) and np.isnan(row['SNR']):df=df.drop(i)
print(len(df),'rows with constraints.')        
display(df)

    

Observation targets files are: ['../Data/ObservationObjectives-9-May-22.csv', '../Data/ObservationObjectives-21-Jun-22.csv', '../Data/ObservationObjectives-09-09-22.csv', '../Data/ObservationObjectives-15-Jul-22.csv', '../Data/ObservationObjectives-09-13-22.csv', '../Data/ObservationObjectives-2022-09-26.csv']
Observation targets file is: ../Data/ObservationObjectives-2022-09-26.csv
141 rows read from file.
85 rows with a potential observation label.
82 rows with masses.
79 rows with redshift.
74 rows with constraints.


,Source class,Source label,Source specification,Distance/redshift (dL in kpc),Sub-specs,freq,Required precision,provisional POC label,Achievable by Concept X?,science motivation,...,"['1.1.b', '1.2.b', '1.3.b', '2.2.b', '3.1.a', '3.2.a', '3.3.d', '4.1.a', '4.1.b', '5.1.a', '5.1.b', '5.1.c', '5.1.d', '5.2.a', '5.2.b', '5.3.a', '5.3.b', '5.4.a', '5.4.b', '5.5.b', '5.7.a']",m1,m2,z,dist,timecut,SNR,CW,f0,resolution
1,1. Stellar-scale mergers,NS binary,1.5 MSun + 1.5 MSun,z = 0.05; dL = 2.30e5,month to week before merger,0.5 Hz,1 arcmin,1.1.b,NaN,standard sirens for cosmological structure; fo...,...,NaN,1.5,1.5,0.05,2.245822e+05,15.0,NaN,False,NaN,60.0
2,1. Stellar-scale mergers,NS binary,1.5 MSun + 1.5 MSun,z = 1.0; dL = 6.79e6,chirping,0.5 Hz,30 arcsec,1.1.c,NaN,standard sirens for cosmological structure; fo...,...,NaN,1.5,1.5,1.00,6.726141e+06,NaN,NaN,False,NaN,30.0
3,1. Stellar-scale mergers,NS binary,1.5 MSun + 1.5 MSun,z = 20; dL = 2.31e8,chirping,0.5 Hz,6 arcsec,1.1.d,NaN,standard sirens for cosmological structure; fo...,...,NaN,1.5,1.5,20.00,2.316106e+08,NaN,NaN,False,NaN,6.0
4,1. Stellar-scale mergers,NS binary,1.5 MSun + 1.5 MSun,z = 10; dL = 1.07e8,chirping,0.5 Hz,1 arcsec,1.1.e,NaN,standard sirens for cosmological structure; fo...,...,NaN,1.5,1.5,10.00,1.065544e+08,NaN,NaN,False,NaN,1.0
5,1. Stellar-scale mergers,NS binary,1.5 MSun + 1.5 MSun,z = 0.15; dL = 6.0e5,days before merger,0.5 Hz,1 deg,1.1.f,NaN,H0 measurements using joint GW-EM detections o...,...,NaN,1.5,1.5,0.15,7.212455e+05,3.0,NaN,False,NaN,3600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,5. IMBH binaries,IMBH + SOBH,5e3 Msun + 1 Msun,z = 0.3; dL = 1.57e6,SNR = 50,0.01-1 Hz (ish),1 arcsec,5.6.a,NaN,H0 measurements associating GW sources with ho...,...,NaN,5000.0,1.0,0.30,1.572542e+06,NaN,50.0,False,NaN,1.0
88,5. IMBH binaries,IMBH + SOBH,5e3 Msun + 1 Msun,z = 1.0; dL = 6.79e6,no particular timescale,0.01-1 Hz (ish),1 arcsec,5.6.b,NaN,H0 measurements associating GW sources with ho...,...,NaN,5000.0,1.0,1.00,6.726141e+06,NaN,NaN,False,NaN,1.0
89,5. IMBH binaries,MBH + IMBH,1e6 MSun + 1e3 MSun,z = 5.5; dL = 5.33e7,SNR = 20,0.01-1 Hz (ish),1 arcsec,5.7.a,NaN,H0 measurements associating GW sources with ho...,...,NaN,1000000.0,1000.0,5.50,5.346936e+07,NaN,20.0,False,NaN,1.0
90,5. IMBH binaries,MBH + IMBH,1e6 MSun + 1e3 MSun,z = 1.0; dL = 6.79e6,no particular timescale,0.01-1 Hz (ish),1 arcsec,5.7.b,NaN,H0 measurements associating GW sources with ho...,...,NaN,1000000.0,1000.0,1.00,6.726141e+06,NaN,NaN,False,NaN,1.0


In [4]:
minidf=df[['Sub-specs','timecut']]
display(minidf.drop_duplicates())

,Sub-specs,timecut
1,month to week before merger,15.0
2,chirping,NaN
5,days before merger,3.0
6,no particular timescale,NaN
7,chirping or capability to detect higher-order ...,NaN
24,year to month before merger,180.0
30,week before merger,7.0
32,post merger,NaN
34,1 day < merger,1.0
36,close to merger or chirping,NaN


In [5]:
# make source dictionaries
objectiveList=[]
minimumSNR=5
for i,row in df.iterrows():
    obj={}
    obj['label']=row['provisional POC label']
    obj['m1']=row['m1']*(1+row['z'])#convert to redshifted mass
    obj['m2']=row['m2']*(1+row['z'])#convert to redshifted mass
    dist=cosmo.luminosity_distance(row['z']).to(u.kpc).value
    obj['dl']=dist
    if row['CW']:
        obj['type']='CW'
        obj['f0']=row['f0']
    else:
        obj['type']='chirp'
        if np.isfinite(row['timecut']):
            obj['timecut']=row['timecut']
    #Now add the conditions
    conditions={}
    if np.isfinite(row['resolution']):
        conditions['resolution']=row['resolution']/3600/180*np.pi #convert arcsec to rad
    snr=minimumSNR
    if np.isfinite(row['SNR']): snr=max([minimumSNR,row['SNR']])
    conditions['SNR']=snr
    obj['conditions']=conditions
    objectiveList.append(obj)
    

### Perform the observations and assessment
We first compute the SNR and then the angular resolution

In [6]:
import importlib
importlib.reload(metrics)
importlib.reload(sources)
addOldRes=False
for mission in missions:
    print('\nProcessing mission',mission['label'],'\n')
    obs_versions=[]
    if addOldRes: versions=[False,True]
    else: versions=[False]
    for useOldRes in versions:
        obs = list()
        for s in objectiveList:
            print('\nProcessing:',s.get('label','--unnamed--'))
            print(s)
            if useOldRes:
                sgn=-1;
                if s['type']=='CW': sgn=1 #There is currently a pathology with getSourceSnr, it needs T>0 for CW and T<0 for chirp
                ob = metrics.getSourceSnr(s,mission,sgn*4.0*constants.year)
                ob = metrics.getResolution(ob)
            else:
                #print('OLD')
                #ob = metrics.getSourceSnr(s,mission,-4.0*constants.year)
                #print('NEW')
                ob = metrics.getSNRandSkyResolution(s,mission,Tmax=4)
            print('SNR,res',ob['SNR'],ob['Angular Resolution'])
            #Perform the assessment
            #The assessment is based on the ratio of the estimated capability to the objective for each of
            #resolution and SNR, with <=1 meaning that the objective is met
            #A total is based on the worst of those present
            conditions=s['conditions']
            assessment={}
            alleps=[]
            print(conditions)
            if 'SNR' in conditions:
                #eps=ob['SNR']/conditions['SNR']
                eps=conditions['SNR']/ob['SNR']
                assessment['eps_SNR']=eps
                alleps.append(eps)
            if 'resolution' in conditions:
                #eps=conditions['resolution']/ob['Angular Resolution']
                eps=ob['Angular Resolution']/conditions['resolution']
                #print(ob['Angular Resolution'][-1],conditions['resolution'],eps)
                assessment['eps_res']=eps
                alleps.append(eps)
            #print('alleps',alleps)
            if len(alleps)>0: 
                assessment['eps']=max(alleps)*(1+sum(alleps)*0) #The last factor ensures that NANs stay NANs
            ob['assessment']=assessment    
            print(assessment)
            obs.append(ob)
        obs_versions.append(obs)
        
    mission['obs']=obs_versions[0]
    if addOldRes:mission['obs_old']=obs_versions[1]




Processing mission LISA(CBE) 


Processing: 1.1.b
{'label': '1.1.b', 'm1': 1.5750000000000002, 'm2': 1.5750000000000002, 'dl': 224582.2244309782, 'type': 'chirp', 'timecut': 15.0, 'conditions': {'resolution': 0.0002908882086657216, 'SNR': 5}}
rough tstart = -126232599.05418241 0.1106704277470008 < f < 4957.7802371184
ncut= 23 n= 977
tstart = -126232599.05418241 t[0]= -129649087.31673475 0.14161770601131066 < f < 4957.7802371184
Recomputing chirp because net resolution is low. 163 < 500.0
rough tstart = -126232599.05418241 0.1106704277470008 < f < 0.9370006802648263
ncut= 119 n= 881
tstart = -126232599.05418241 t[0]= -126938305.0432465 0.14273792731701163 < f < 0.9370006802648263
SNR,res 0.039472664930598915 0.053588239371690376
{'resolution': 0.0002908882086657216, 'SNR': 5}
{'eps_SNR': 126.66993750715922, 'eps_res': 184.22279685207894, 'eps': 184.22279685207894}

Processing: 1.1.c
{'label': '1.1.c', 'm1': 3.0, 'm2': 3.0, 'dl': 6726141.242139786, 'type': 'chirp', 'conditions': {'resol

ncut= 52 n= 948
tstart = -126232599.05418241 t[0]= -126524434.99375623 1.9157110357027627e-05 < f < 0.00015770744607030058
SNR,res 31.609707808717673 1.908631662882427
{'resolution': 0.0002908882086657216, 'SNR': 5}
{'eps_SNR': 0.1581792539892142, 'eps_res': 6561.392335691952, 'eps': 6561.392335691952}

Processing: 2.2.b
{'label': '2.2.b', 'm1': 1500000.0, 'm2': 1500000.0, 'dl': 15875191.123148063, 'type': 'chirp', 'conditions': {'resolution': 0.0002908882086657216, 'SNR': 5}}
rough tstart = -126232599.05418241 2.041990409488994e-05 < f < 0.005205669248974321
ncut= 22 n= 978
tstart = -126232599.05418241 t[0]= -126238887.01450107 2.3070021301879525e-05 < f < 0.005205669248974321
SNR,res 4727.094978480198 0.0021693715897497692
{'resolution': 0.0002908882086657216, 'SNR': 5}
{'eps_SNR': 0.0010577320791653616, 'eps_res': 7.45775017729486, 'eps': 7.45775017729486}

Processing: 2.3.b
{'label': '2.3.b', 'm1': 1500000.0, 'm2': 1500000.0, 'dl': 15875191.123148063, 'type': 'chirp', 'timecut': 1.

tstart = -126232599.05418241 t[0]= -127134159.12046073 0.017312685690939026 < f < 48.30349035300498
SNR,res 0.6050939193295997 0.023972872092890743
{'resolution': 4.84813681109536e-06, 'SNR': 5}
{'eps_SNR': 8.263180045735114, 'eps_res': 4944.759817426532, 'eps': 4944.759817426532}

Processing: 5.2.a
{'label': '5.2.a', 'm1': 6000.0, 'm2': 120.0, 'dl': 15875191.123148063, 'type': 'chirp', 'conditions': {'resolution': 4.84813681109536e-06, 'SNR': 5}}
rough tstart = -126232599.05418241 0.0025555801517474675 < f < 1.156161770734939
ncut= 55 n= 945
tstart = -126232599.05418241 t[0]= -126712100.6211227 0.0035784075970976144 < f < 1.156161770734939
SNR,res 7.539053950610109 0.007556031847903863
{'resolution': 4.84813681109536e-06, 'SNR': 5}
{'eps_SNR': 0.6632131873250977, 'eps_res': 1558.5434451047797, 'eps': 1558.5434451047797}

Processing: 5.2.b
{'label': '5.2.b', 'm1': 18000.0, 'm2': 360.0, 'dl': 82565701.74638993, 'type': 'chirp', 'conditions': {'resolution': 4.84813681109536e-06, 'SNR': 5

tstart = -126232599.05418241 t[0]= -127075144.8242819 1.912745198577301e-05 < f < 0.003904251936730741
Recomputing chirp because net resolution is low. 141 < 500.0
rough tstart = -126232599.05418241 1.7063120383781664e-05 < f < 4.862477070438777e-05
ncut= 111 n= 889
tstart = -126232599.05418241 t[0]= -126310905.00903544 1.9168657139555096e-05 < f < 4.862477070438777e-05
SNR,res 2.9098258982713894 4.047418851811325
{'resolution': 0.0002908882086657216, 'SNR': 5}
{'eps_SNR': 1.7183158631484787, 'eps_res': 13914.00108782847, 'eps': 13914.00108782847}

Processing: 2.1.c
{'label': '2.1.c', 'm1': 2000000.0, 'm2': 2000000.0, 'dl': 6726141.242139786, 'type': 'chirp', 'conditions': {'resolution': 0.0001454441043328608, 'SNR': 5}}
rough tstart = -126232599.05418241 1.7063120383781664e-05 < f < 0.003904251936730741
ncut= 21 n= 979
tstart = -126232599.05418241 t[0]= -127075144.8242819 1.912745198577301e-05 < f < 0.003904251936730741
SNR,res 16692.28425612247 1.2869842099346084e-05
{'resolution': 0

SNR,res 10.661832344415277 0.0037982274642872384
{'resolution': 4.84813681109536e-06, 'SNR': 5}
{'eps_SNR': 0.46896254212992067, 'eps_res': 783.4406520036073, 'eps': 783.4406520036073}

Processing: 5.2.b
{'label': '5.2.b', 'm1': 18000.0, 'm2': 360.0, 'dl': 82565701.74638993, 'type': 'chirp', 'conditions': {'resolution': 4.84813681109536e-06, 'SNR': 5}}
rough tstart = -126232599.05418241 0.001286718425527953 < f < 0.3853872569116464
ncut= 54 n= 946
tstart = -126232599.05418241 t[0]= -126639293.74981229 0.0017512321718563962 < f < 0.3853872569116464
SNR,res 5.972798882855355 0.009831310304298176
{'resolution': 4.84813681109536e-06, 'SNR': 5}
{'eps_SNR': 0.8371284716035342, 'eps_res': 2027.8533150711453, 'eps': 2027.8533150711453}

Processing: 5.2.d
{'label': '5.2.d', 'm1': 4000.0, 'm2': 80.0, 'dl': 6726141.242139786, 'type': 'chirp', 'conditions': {'resolution': 4.84813681109536e-06, 'SNR': 5}}
rough tstart = -126232599.05418241 0.003292115314911464 < f < 1.7342426561024087
ncut= 55 n= 9

...

tstart = -126232599.05418241 t[0]= -126712100.6211227 0.0035784075970976144 < f < 1.156161770734939
SNR,res 23.780149806373565 0.0025755047759738435
{'resolution': 4.84813681109536e-06, 'SNR': 5}
{'eps_SNR': 0.210259398730108, 'eps_res': 531.2359936047161, 'eps': 531.2359936047161}

Processing: 5.2.b
{'label': '5.2.b', 'm1': 18000.0, 'm2': 360.0, 'dl': 82565701.74638993, 'type': 'chirp', 'conditions': {'resolution': 4.84813681109536e-06, 'SNR': 5}}
rough tstart = -126232599.05418241 0.001286718425527953 < f < 0.3853872569116464
ncut= 54 n= 946
tstart = -126232599.05418241 t[0]= -126639293.74981229 0.0017512321718563962 < f < 0.3853872569116464
SNR,res 11.687063272855244 0.021525355666727156
{'resolution': 4.84813681109536e-06, 'SNR': 5}
{'eps_SNR': 0.42782347312289853, 'eps_res': 4439.923315997315, 'eps': 4439.923315997315}

Processing: 5.2.d
{'label': '5.2.d', 'm1': 4000.0, 'm2': 80.0, 'dl': 6726141.242139786, 'type': 'chirp', 'conditions': {'resolution': 4.84813681109536e-06, 'SNR': 

tstart = -126232599.05418241 t[0]= -127075144.8242819 1.912745198577301e-05 < f < 0.003904251936730741
SNR,res 19721.686579961093 1.0419897843731642e-05
{'resolution': 4.84813681109536e-06, 'SNR': 5}
{'eps_SNR': 0.0002535280124104814, 'eps_res': 2.149258209851844, 'eps': 2.149258209851844}

Processing: 2.1.g
{'label': '2.1.g', 'm1': 2000000.0, 'm2': 2000000.0, 'dl': 6726141.242139786, 'type': 'chirp', 'timecut': 7.0, 'conditions': {'resolution': 0.0002908882086657216, 'SNR': 5}}
rough tstart = -126232599.05418241 1.7063120383781664e-05 < f < 0.003904251936730741
ncut= 21 n= 979
tstart = -126232599.05418241 t[0]= -127075144.8242819 1.912745198577301e-05 < f < 0.003904251936730741
Recomputing chirp because net resolution is low. 362 < 500.0
rough tstart = -126232599.05418241 1.7063120383781664e-05 < f < 0.00015770744607030058
ncut= 52 n= 948
tstart = -126232599.05418241 t[0]= -126524434.99375623 1.9157110357027627e-05 < f < 0.00015770744607030058
SNR,res 44.69621875440317 0.1294214198625

SNR,res 42.04765234242225 0.000540925244080465
{'resolution': 4.84813681109536e-06, 'SNR': 5}
{'eps_SNR': 0.1189127031226772, 'eps_res': 111.57384066442042, 'eps': 111.57384066442042}

Processing: 5.3.b
{'label': '5.3.b', 'm1': 9000.0, 'm2': 900.0, 'dl': 82565701.74638993, 'type': 'chirp', 'conditions': {'resolution': 4.84813681109536e-06, 'SNR': 5}}
rough tstart = -126232599.05418241 0.0010948764051370078 < f < 1.073410896733444
ncut= 48 n= 952
tstart = -126232599.05418241 t[0]= -126279391.55275586 0.0015243899294297768 < f < 1.073410896733444
SNR,res 20.502595094839023 0.001188494835221957
{'resolution': 4.84813681109536e-06, 'SNR': 5}
{'eps_SNR': 0.24387156732459764, 'eps_res': 245.14465691273165, 'eps': 245.14465691273165}

Processing: 5.3.d
{'label': '5.3.d', 'm1': 2000.0, 'm2': 200.0, 'dl': 6726141.242139786, 'type': 'chirp', 'conditions': {'resolution': 4.84813681109536e-06, 'SNR': 5}}
rough tstart = -126232599.05418241 0.0028012655753002513 < f < 4.8303490353004985
ncut= 47 n= 

tstart = -126232599.05418241 t[0]= -127075144.8242819 1.912745198577301e-05 < f < 0.003904251936730741
Recomputing chirp because net resolution is low. 141 < 500.0
rough tstart = -126232599.05418241 1.7063120383781664e-05 < f < 4.862477070438777e-05
ncut= 111 n= 889
tstart = -126232599.05418241 t[0]= -126310905.00903544 1.9168657139555096e-05 < f < 4.862477070438777e-05
SNR,res 2.9097803783965634 0.2129937804730035
{'resolution': 0.0002908882086657216, 'SNR': 5}
{'eps_SNR': 1.7183427440511003, 'eps_res': 732.218681018344, 'eps': 732.218681018344}

Processing: 2.1.c
{'label': '2.1.c', 'm1': 2000000.0, 'm2': 2000000.0, 'dl': 6726141.242139786, 'type': 'chirp', 'conditions': {'resolution': 0.0001454441043328608, 'SNR': 5}}
rough tstart = -126232599.05418241 1.7063120383781664e-05 < f < 0.003904251936730741
ncut= 21 n= 979
tstart = -126232599.05418241 t[0]= -127075144.8242819 1.912745198577301e-05 < f < 0.003904251936730741
SNR,res 19721.686579961093 3.4732992815836443e-07
{'resolution': 0

tstart = -126232599.05418241 t[0]= -128510960.35161458 0.003977662185665147 < f < 4.8303490353004985
SNR,res 69.5156699365171 1.1234605611054165e-05
{'resolution': 4.84813681109536e-06, 'SNR': 5}
{'eps_SNR': 0.07192622907275562, 'eps_res': 2.317303749626629, 'eps': 2.317303749626629}

Processing: 5.4.a
{'label': '5.4.a', 'm1': 3000.0, 'm2': 3000.0, 'dl': 15875191.123148063, 'type': 'chirp', 'conditions': {'resolution': 4.84813681109536e-06, 'SNR': 5}}
rough tstart = -126232599.05418241 0.000988419149278962 < f < 2.6028346244871603
ncut= 27 n= 973
tstart = -126232599.05418241 t[0]= -127450252.48622383 0.001222889576509801 < f < 2.6028346244871603
SNR,res 122.29148386346478 6.611544648497174e-06
{'resolution': 4.84813681109536e-06, 'SNR': 5}
{'eps_SNR': 0.04088592142346043, 'eps_res': 1.3637289759162965, 'eps': 1.3637289759162965}

Processing: 5.4.b
{'label': '5.4.b', 'm1': 9000.0, 'm2': 9000.0, 'dl': 82565701.74638993, 'type': 'chirp', 'conditions': {'resolution': 4.84813681109536e-06, 

### Show assessments

In [7]:
#Make a dataframe with the results
objectiveNames=df['provisional POC label'].values
if True:
    #by hand list of those which were not used in the STM as of 9/12/2022
    unused=['1.1.b', '1.2.b', '1.3.b', '2.2.b', '3.1.a', '3.2.a', '3.3.d', '4.1.a', '4.1.b', '5.1.a', '5.1.b', '5.1.c', '5.1.d', '5.2.a', '5.2.b', '5.3.a', '5.3.b', '5.4.a', '5.4.b', '5.5.b', '5.7.a']
    for i in range(len(objectiveNames)):
        oname=objectiveNames[i]
        if oname in unused: objectiveNames[i]='XX'+oname+'XX'
nm=len(missions)
colNames=missionNames
if addOldRes:nm=nm*2+1
data=np.zeros((len(obs),nm))
for i,mission in enumerate(missions):
    for j,ob in enumerate(mission['obs']):
        #print(missionNames[i],objectiveNames[j],ob['assessment'])
        data[j,i]=ob['assessment'].get('eps',None)
if addOldRes:
    colNames=list(colNames)+['<--NEW | OLD-->']+[name+"'" for name in missionNames]
    data[:,len(missions)]=float('nan')
    for i,mission in enumerate(missions):
        for j,ob in enumerate(mission['obs_old']):
            #print(missionNames[i],objectiveNames[j],ob['assessment'])
            data[j,i+len(missions)+1]=ob['assessment'].get('eps',None)


adf = pd.DataFrame(data,
                  index=objectiveNames,
                  columns=colNames)

    
#adf=adf.dropna()
def shrink():
    return [dict(selector="th",
                 props=[("font-size", "6pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")])
]
def zoom():
    return [
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]
#pd.options.display.precision=2
s=adf.style.background_gradient(vmin=0.1,vmax=10,cmap='RdYlGn_r').set_properties(**{'max-width': '120px', 'font-size': '6pt'})
#s=s.set_table_styles(shrink())
s=s.set_table_styles(shrink()+zoom())
s=s.set_caption("Net performance")
s=s.format('{:.2g}')
display(s)
if True:
    dates = date.today().strftime("%Y_%m_%d")
    outfile=datadir+'ScoredObservations_net_'+dates+'.csv'
    #adf.to_csv(outfile,index=False)
    adf.to_csv(outfile,index=True)
    print('Wrote to file:',outfile)

,LISACBE,TwinLISA,LISAGrande,LISAU,GoBIGLISA,ALIA,ALIAtwin,GoBIGALIA
XX1.1.bXX,1.8e+02,1.1e+02,2.1e+02,4.9e+02,1.3e+02,10,5.8,5.1
1.1.c,4.7e+03,2.7e+03,5.4e+03,1.1e+04,1.2e+03,2.8e+02,1.6e+02,53
1.1.d,5e+04,2.9e+04,4.8e+04,1e+05,2.2e+03,5.8e+03,3.1e+03,1.6e+02
1.1.e,2.8e+05,1.6e+05,3e+05,6.3e+05,1.3e+04,2.7e+04,1.5e+04,8e+02
1.1.f,3.4e+02,2.4e+02,4e+02,1.3e+03,3.6e+02,20,14,14
1.1.g,1.4e+05,8.1e+04,1.6e+05,3.4e+05,6.9e+03,8.3e+03,4.8e+03,2.7e+02
1.1.h,2.3e+03,1.3e+03,2.7e+03,5.6e+03,1.2e+03,1.4e+02,80,53
1.1.i,1.1e+03,8.1e+02,1.3e+03,4.4e+03,1.2e+03,76,53,53
XX1.2.bXX,2.6e+02,1.5e+02,3e+02,6.1e+02,1.1e+02,16,9.2,5.2
1.2.c,69,49,80,2.6e+02,72,4.3,3.1,3.1


Wrote to file: ../Data/ScoredObservations_net_2022_09_26.csv


In [8]:
#Make a dataframe with the results
objectiveNames=df['provisional POC label'].values
nm=len(missions)
colNames=missionNames
if addOldRes:nm=nm*2+1
data=np.zeros((len(obs),nm))
for i,mission in enumerate(missions):
    for j,ob in enumerate(mission['obs']):
        #print(missionNames[i],objectiveNames[j],ob['assessment'])
        data[j,i]=ob['assessment'].get('eps_SNR',None)
if addOldRes:
    colNames=list(colNames)+['<--NEW | OLD-->']+[name+"'" for name in missionNames]
    data[:,len(missions)]=float('nan')
    for i,mission in enumerate(missions):
        for j,ob in enumerate(mission['obs_old']):
            #print(missionNames[i],objectiveNames[j],ob['assessment'])
            data[j,i+len(missions)+1]=ob['assessment'].get('eps_SNR',None)


adf = pd.DataFrame(data,
                  index=objectiveNames,
                  columns=colNames)

    
#adf=adf.dropna()
def shrink():
    return [dict(selector="th",
                 props=[("font-size", "6pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")])
]
def zoom():
    return [
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]
#pd.options.display.precision=2
s=adf.style.background_gradient(vmin=0.1,vmax=10,cmap='RdYlGn_r').set_properties(**{'max-width': '120px', 'font-size': '6pt'})
#s=s.set_table_styles(shrink())
s=s.set_table_styles(shrink()+zoom())
s=s.set_caption("SNR performance")
s=s.format('{:.2g}')
display(s)
if True:
    dates = date.today().strftime("%Y_%m_%d")
    outfile=datadir+'ScoredObservations_SNR_'+dates+'.csv'
    #adf.to_csv(outfile,index=False)
    adf.to_csv(outfile,index=True)
    print('Wrote to file:',outfile)



,LISACBE,TwinLISA,LISAGrande,LISAU,GoBIGLISA,ALIA,ALIAtwin,GoBIGALIA
XX1.1.bXX,1.3e+02,90,1.5e+02,4.9e+02,1.3e+02,7.3,5.1,5.1
1.1.c,1.1e+03,8.1e+02,1.3e+03,4.4e+03,1.2e+03,76,53,53
1.1.d,6.1e+02,4.3e+02,5.5e+02,1.8e+03,5.3e+02,88,62,62
1.1.e,8.4e+02,5.9e+02,8.5e+02,2.8e+03,7.9e+02,97,69,69
1.1.f,3.4e+02,2.4e+02,4e+02,1.3e+03,3.6e+02,20,14,14
1.1.g,1.1e+03,8.1e+02,1.3e+03,4.4e+03,1.2e+03,76,53,53
1.1.h,1.1e+03,8.1e+02,1.3e+03,4.4e+03,1.2e+03,76,53,53
1.1.i,1.1e+03,8.1e+02,1.3e+03,4.4e+03,1.2e+03,76,53,53
XX1.2.bXX,1.1e+02,76,1.2e+02,4.1e+02,1.1e+02,7.4,5.2,5.2
1.2.c,69,49,80,2.6e+02,72,4.3,3.1,3.1


Wrote to file: ../Data/ScoredObservations_SNR_2022_09_26.csv


In [9]:
#Make a dataframe with the results
objectiveNames=df['provisional POC label'].values
nm=len(missions)
colNames=missionNames
if addOldRes:nm=nm*2+1
data=np.zeros((len(obs),nm))
for i,mission in enumerate(missions):
    for j,ob in enumerate(mission['obs']):
        #print(missionNames[i],objectiveNames[j],ob['assessment'])
        data[j,i]=ob['assessment'].get('eps_res',None)
if addOldRes:
    colNames=list(colNames)+['<--NEW | OLD-->']+[name+"'" for name in missionNames]
    data[:,len(missions)]=float('nan')
    for i,mission in enumerate(missions):
        for j,ob in enumerate(mission['obs_old']):
            #print(missionNames[i],objectiveNames[j],ob['assessment'])
            data[j,i+len(missions)+1]=ob['assessment'].get('eps_res',None)


adf = pd.DataFrame(data,
                  index=objectiveNames,
                  columns=colNames)

    
#adf=adf.dropna()
def shrink():
    return [dict(selector="th",
                 props=[("font-size", "6pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")])
]
def zoom():
    return [
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]
#pd.options.display.precision=2
s=adf.style.background_gradient(vmin=0.1,vmax=10,cmap='RdYlGn_r').set_properties(**{'max-width': '120px', 'font-size': '6pt'})
#s=s.set_table_styles(shrink())
s=s.set_table_styles(shrink()+zoom())
s=s.set_caption("Resolution performance")
s=s.format('{:.2g}')
display(s)
if True:
    dates = date.today().strftime("%Y_%m_%d")
    outfile=datadir+'ScoredObservations_res_'+dates+'.csv'
    adf.to_csv(outfile,index=True)
    print('Wrote to file:',outfile)

,LISACBE,TwinLISA,LISAGrande,LISAU,GoBIGLISA,ALIA,ALIAtwin,GoBIGALIA
XX1.1.bXX,1.8e+02,1.1e+02,2.1e+02,4.4e+02,9,10,5.8,0.33
1.1.c,4.7e+03,2.7e+03,5.4e+03,1.1e+04,2.3e+02,2.8e+02,1.6e+02,9.1
1.1.d,5e+04,2.9e+04,4.8e+04,1e+05,2.2e+03,5.8e+03,3.1e+03,1.6e+02
1.1.e,2.8e+05,1.6e+05,3e+05,6.3e+05,1.3e+04,2.7e+04,1.5e+04,8e+02
1.1.f,8.4,4.9,9.8,20,0.42,0.46,0.27,0.015
1.1.g,1.4e+05,8.1e+04,1.6e+05,3.4e+05,6.9e+03,8.3e+03,4.8e+03,2.7e+02
1.1.h,2.3e+03,1.3e+03,2.7e+03,5.6e+03,1.1e+02,1.4e+02,80,4.6
1.1.i,39,22,45,94,1.9,2.3,1.3,0.076
XX1.2.bXX,2.6e+02,1.5e+02,3e+02,6.1e+02,12,16,9.2,0.53
1.2.c,2.1,1.2,2.5,5.2,0.11,0.12,0.071,0.0041


Wrote to file: ../Data/ScoredObservations_res_2022_09_26.csv
